In [ ]:
import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd


sys.path.append("../libs")
from classification import label_wav
from classification import input_data
from classification import models
from tensorflow.python.platform import gfile

In [ ]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','../data/raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','../summaries','Summaries Folder')
flags.DEFINE_string('results_dir','../results','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','../models','Models Folder')
flags.DEFINE_string('predictions_dir','../predictions','Predictions Directory')
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
FLAGS=flags.FLAGS

In [ ]:
model_architecture='c_rnn'
model_version='30000'

In [ ]:
test_dir=os.path.join(FLAGS.data_dir,'test','audio','*.wav')

In [ ]:
# Must be in sorted order
testing_files_list=sorted(gfile.Glob(test_dir))

In [ ]:
labels_list=input_data.prepare_words_list(FLAGS.wanted_words.split(','))
graph=os.path.join(FLAGS.models_dir,model_architecture,'ckpt-'+model_version+'-small-batched.pb')

In [ ]:
pred_mat,pred=label_wav.label_wav_batched(testing_files_list,labels_list,graph,batch_size=1000)

In [ ]:
save_folder='c_rnn_output'

In [ ]:
save_path=os.path.join(FLAGS.predictions_dir,save_folder)
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
f_names=[]
for f in testing_files_list:
    f_names.append(os.path.basename(f))
pred_df=pd.DataFrame({'fname':f_names,'label':pred})

In [ ]:
pred_df.to_csv(os.path.join(save_path,'predictions.csv'),index=None)
np.save(os.path.join(save_path,'pred_mat.npy'),pred_mat)
pred_df

In [ ]:
testing_files_list=''
for f in f_names:
    testing_files_list=testing_files_list+str(os.path.join('test','audio',f))+'\n'